In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# Przygotowanie zestawu danych
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Definicja modelu
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv3(x))
        x = nn.functional.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        output = nn.functional.log_softmax(x, dim=1)
        return output

# Definicja funkcji straty i optymalizatora
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Trenowanie modelu
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0

# Testowanie modelu
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {100 * correct / total:.2f}%")



  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
[1, 100] loss: 2.289
[1, 200] loss: 2.092
[1, 300] loss: 1.966
[1, 400] loss: 1.845
[1, 500] loss: 1.799
[1, 600] loss: 1.681
[1, 700] loss: 1.643
[2, 100] loss: 1.564
[2, 200] loss: 1.526
[2, 300] loss: 1.504
[2, 400] loss: 1.462
[2, 500] loss: 1.415
[2, 600] loss: 1.427
[2, 700] loss: 1.373
[3, 100] loss: 1.323
[3, 200] loss: 1.311
[3, 300] loss: 1.312
[3, 400] loss: 1.257
[3, 500] loss: 1.218
[3, 600] loss: 1.207
[3, 700] loss: 1.231
[4, 100] loss: 1.163
[4, 200] loss: 1.133
[4, 300] loss: 1.110
[4, 400] loss: 1.121
[4, 500] loss: 1.109
[4, 600] loss: 1.095
[4, 700] loss: 1.086
[5, 100] loss: 1.059
[5, 200] loss: 1.029
[5, 300] loss: 1.030
[5, 400] loss: 0.995
[5, 500] loss: 1.021
[5, 600] loss: 1.026
[5, 700] loss: 0.996
[6, 100] loss: 0.967
[6, 200] loss: 0.966
[6, 300] loss: 0.964
[6, 400] loss: 0.972
[6, 500] loss: 0.940
[6, 600] loss: 0.912
[6, 700] loss: 0.932
[7, 100] loss: 0.907
[7, 200]